In [1]:
import math
import pandas as pd
import numpy as np

class PBChromBatchMode(object):
  
    def __init__(self,MassIn,VolumeIn,ResinID,nCOL,nCYC,BedHeight,LinVeloc,Disposable,TrainPosition,Batches):
        self.MassIn = MassIn
        self.VolumeIn = VolumeIn
        self.nCOL = nCOL
        self.nCYC = nCYC
        self.BedHeight = BedHeight
        self.ResinID = ResinID
        self.LinVeloc = LinVeloc
        self.Disposable = Disposable
        self.TrainPosition = TrainPosition
        self.Batches = Batches
        #Load Resin Database
        ResinData=pd.read_excel('Database.xlsx',sheetname='ResinList',index_col=0)        
        #Check if resin exists, if its position is correct and if the linear velocity is acceptable
        if self.ResinID<=ResinData.shape[0] and self.TrainPosition==ResinData.get_value(self.ResinID,'Position') and self.LinVeloc<=ResinData.get_value(self.ResinID,'Linear Velocity (cm/hr)'):
            pass
        else:
            raise ValueError
    
    def Sizing(self):
        #Check column technology option and select available list of diameters. it would be better to read form database
        if self.Disposable==True:
            ActualDiameters=np.array([5,8,10,14,20,25,30,45,60])
        else:
            ActualDiameters=np.array([5,8,10,14,20,25,30,45,60,80,100,110,120,140,160,180,200])
        #Load Resin Database
        ResinData=pd.read_excel('Database.xlsx',sheetname='ResinList',index_col=0)
        #Load SOPs from Database
        Protocol=pd.read_excel('Database.xlsx',sheetname='SOPs',index_col=0)
        #find DBC for the selected resin
        DBC=ResinData.get_value(self.ResinID,'DBC (g/L)')
        #identify resin lifespan
        ResinLife=ResinData.get_value(self.ResinID,'Resin Lifespan (#cycles)')
        #identify elution pool volume in CVs
        ElutionPool=ResinData.get_value(self.ResinID,'Elution CVs')
        #Estimate CV
        ColumnVolume=self.MassIn*1000/(DBC*self.nCOL*self.nCYC)
        #Estimate Diameter
        ColumnDiameter=round(math.sqrt(ColumnVolume*4000/(math.pi*self.BedHeight)),0)
        #Correct to find an available diameter
        idx=(np.abs(ActualDiameters-ColumnDiameter)).argmin()
        CorrectedDiameter=ActualDiameters[idx]
        #Correct the number of cycles per batch
        CorrectedNCYC=round(4000*1000*self.MassIn/(DBC*self.nCOL*math.pi*self.BedHeight*CorrectedDiameter**2),0)
        #Correct the bed-height
        CorrectedBedHeight=round(4000*1000*self.MassIn/(DBC*self.nCOL*math.pi*CorrectedNCYC*CorrectedDiameter**2),1)
        #Re-calculate the volume of the column
        CorrectedColumnVolume=math.pi*CorrectedBedHeight*CorrectedDiameter**2/4000
        #Check by how much the bed-height was changed
        BedHeightDifference=abs((CorrectedBedHeight-self.BedHeight)/self.BedHeight)
        #Calculate number of self-packed column packings OR the number of pre-packed columns per campaign
        ColumnActivity=math.ceil(CorrectedNCYC*self.Batches/ResinLife)
        #Below buffer that occur in every cycle of every batch
        #Equilibration buffer volume
        EquilBuffer=Protocol.get_value('Equilibration (CVs)','Chromatography')*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*CorrectedColumnVolume*CorrectedNCYC*self.nCOL
        #Wash buffer volume
        WashBuffer=Protocol.get_value('Wash (CVs)','Chromatography')*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*CorrectedColumnVolume*CorrectedNCYC*self.nCOL
        #Elution buffer volume
        ElutionBuffer=ElutionPool*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*CorrectedColumnVolume*CorrectedNCYC*self.nCOL
        #Strip buffer volume
        StripBuffer=Protocol.get_value('Strip (CVs)','Chromatography')*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*CorrectedColumnVolume*CorrectedNCYC*self.nCOL
        #Regeneration buffer
        RegenBuffer=Protocol.get_value('Regereration (CVs)','Chromatography')*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*CorrectedColumnVolume*CorrectedNCYC*self.nCOL
        #Below buffer that occur once per batch
        #Sanitisation buffer
        SanitBuffer=Protocol.get_value('Sanitisation (CVs)','Chromatography')*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*CorrectedColumnVolume*self.nCOL
        #Storage buffer
        StorageBuffer=Protocol.get_value('Storage (CVs)','Chromatography')*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*CorrectedColumnVolume*self.nCOL
        #Pre batch operation column testing
        PreOperBuffer=Protocol.get_value('Pre-First Operation (CVs)','Chromatography')*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*CorrectedColumnVolume*self.nCOL
        #Below buffer that occur every time a self-packed chromatography column is packed
        if self.Disposable==False:
            #packing buffer
            PackingBuffer=Protocol.get_value('Packing (CVs)','Chromatography')*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*CorrectedColumnVolume*self.nCOL
            #unpacking buffer
            UnpackBuffer=Protocol.get_value('Unpacking (CVs)','Chromatography')*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*CorrectedColumnVolume*self.nCOL
            #Pre-packing column qualification and testing
            PrePackBuffer=Protocol.get_value('Pre-packing (CVs)','Chromatography')*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*CorrectedColumnVolume*self.nCOL
        else:
            PackingBuffer=0
            UnpackBuffer=0
            PrePackBuffer=0            
        BuffersList=[PreOperBuffer,EquilBuffer,WashBuffer,ElutionBuffer,StripBuffer,RegenBuffer,SanitBuffer,StorageBuffer,PrePackBuffer,PackingBuffer,UnpackBuffer]
        #Check bed-height to be within limits
        if BedHeightDifference<=Protocol.get_value('Bed-height Tolerance','Chromatography'):            
            return CorrectedColumnVolume,CorrectedDiameter,CorrectedBedHeight,CorrectedNCYC,ColumnActivity,DBC,BuffersList
        else:
            return "Bed-height out of limits"
        
    def MassBalance(self):
        #Load Resin Database
        ResinData=pd.read_excel('Database.xlsx',sheetname='ResinList',index_col=0)
        #find Yield for the selected resin
        StepYield=ResinData.get_value(self.ResinID,'Yield')
        #identify elution pool volume in CVs
        ElutionPool=ResinData.get_value(self.ResinID,'Elution CVs')
        #Calculate Mass Out
        MassOut=self.MassIn*StepYield
        #Define mode of operation
        ModeOfOperation=ResinData.get_value(self.ResinID,'Mode of Operation')
        #Bind and Elute mode
        if ModeOfOperation==1:
            #Calculate Volume Out
            VolumeOut=PBChromBatchMode.Sizing(self)[0]*PBChromBatchMode.Sizing(self)[3]*self.nCOL*ElutionPool
        #Flow-through mode
        elif ModeOfOperation==0:
            #Calculate Volume Out
            VolumeOut=self.VolumeIn
        #Calculate Concentration Out
        ConcOut=MassOut*1000/VolumeOut
        return MassOut,VolumeOut,ConcOut,ElutionPool,StepYield
    
    def Timeframe(self):
        #Load SOPs from Database
        Protocol=pd.read_excel('Database.xlsx',sheetname='SOPs',index_col=0)
        #time to prepare
        PrepTime=Protocol.get_value('Preparation Time (hr)','Chromatography')+Protocol.get_value('Pre-First Operation (CVs)','Chromatography')*PBChromBatchMode.Sizing(self)[2]/self.LinVeloc
        #time to operate a single cycle
        EquilTime=Protocol.get_value('Equilibration (CVs)','Chromatography')*PBChromBatchMode.Sizing(self)[2]/self.LinVeloc
        LoadTime=PBChromBatchMode.Sizing(self)[5]/(self.MassIn*1000/self.VolumeIn)*PBChromBatchMode.Sizing(self)[2]/self.LinVeloc
        WashTime=Protocol.get_value('Wash (CVs)','Chromatography')*PBChromBatchMode.Sizing(self)[2]/self.LinVeloc
        ElutionTime=PBChromBatchMode.MassBalance(self)[3]*PBChromBatchMode.Sizing(self)[2]/self.LinVeloc
        CleanTime=(Protocol.get_value('Strip (CVs)','Chromatography')+Protocol.get_value('Regereration (CVs)','Chromatography'))*PBChromBatchMode.Sizing(self)[2]/self.LinVeloc
        #Operating time per batch
        OperTime=(EquilTime+LoadTime+WashTime+ElutionTime+CleanTime)*PBChromBatchMode.Sizing(self)[3]
        #time to sanitise and store
        StoreTime=(Protocol.get_value('Sanitisation (CVs)','Chromatography')+Protocol.get_value('Storage (CVs)','Chromatography'))*PBChromBatchMode.Sizing(self)[2]/self.LinVeloc
        #time to turnaround hold-tanks
        HoldTanksCIPandSIPTime=Protocol.get_value('CIP Time (hr)','Hold-Tanks')+Protocol.get_value('SIP Time (hr)','Hold-Tanks')
        #Check Column technology
        if self.Disposable==False:
            #time to prep for packing and unpacking non-manufacturing
            PackingPrepTime=Protocol.get_value('Packing Prep. Time (hr)','Chromatography')+Protocol.get_value('Unpacking Prep. Time (hr)','Chromatography')
            #time to pack and unpack column packing unpanking buffers
            PackingTime=(Protocol.get_value('Pre-packing (CVs)','Chromatography')+Protocol.get_value('Packing (CVs)','Chromatography'))*PBChromBatchMode.Sizing(self)[2]/self.LinVeloc
            UnpackingTime=Protocol.get_value('Unpacking (CVs)','Chromatography')*PBChromBatchMode.Sizing(self)[2]/self.LinVeloc
        else:
            PackingPrepTime=0
            PackingTime=0
            UnpackingTime=0
        #check if operation is within time limits
        if OperTime<=Protocol.get_value('Duration Limit (hr)','Chromatography'):
            return PackingPrepTime,PrepTime,PackingTime,OperTime,StoreTime,UnpackingTime,HoldTanksCIPandSIPTime
        else:
            return "Operating time for a sinle batch exceeds the duration limit"
    
    
    def EquipmentAndMaterials(self):
        #Load Equipment from Database
        Equipment=pd.read_excel('Database.xlsx',sheetname='Equipment',index_col=0)
        #Load Materials from Database
        Materials=pd.read_excel('Database.xlsx',sheetname='Materials',index_col=0)
        #Load SOPs from Database
        Protocol=pd.read_excel('Database.xlsx',sheetname='SOPs',index_col=0)
        #Calculate Flowrate
        Flowrate=math.pi*PBChromBatchMode.Sizing(self)[1]**2/4000*self.LinVeloc/60
        #Product hold-tank with GF, PW, WFI, CIP
        if PBChromBatchMode.MassBalance(self)[1]/Protocol.get_value('Space Efficiency','Hold-Tanks')>Equipment.get_value('Hold-Tank','Max. Size'):
            NumberOfProductTanks=math.ceil((PBChromBatchMode.MassBalance(self)[1]/Protocol.get_value('Space Efficiency','Hold-Tanks'))/Equipment.get_value('Hold-Tank','Max. Size'))
        else:
            NumberOfProductTanks=1
        ProductTankVolume=(PBChromBatchMode.MassBalance(self)[1]/Protocol.get_value('Space Efficiency','Hold-Tanks'))/NumberOfProductTanks
        if ProductTankVolume<=10:
            CorrectedProductTankVolume=math.ceil(ProductTankVolume)
        elif ProductTankVolume<=100 and ProductTankVolume>10:
            CorrectedProductTankVolume=int(math.ceil(ProductTankVolume/10.0))*10
        elif ProductTankVolume<=1000 and ProductTankVolume>100:
            CorrectedProductTankVolume=int(math.ceil(ProductTankVolume/100.0))*100
        else:
            CorrectedProductTankVolume=int(math.ceil(ProductTankVolume/1000.0))*1000
        ProductHoldTankCost=NumberOfProductTanks*Equipment.get_value('Hold-Tank','Base Cost')*(CorrectedProductTankVolume/Equipment.get_value('Hold-Tank','Base Size'))**Equipment.get_value('Hold-Tank','Scaling Factor')
        ProductHoldTankGuard=NumberOfProductTanks*Materials.get_value('Hold-Tank Guard Filter','Base Cost')*(CorrectedProductTankVolume/Materials.get_value('Hold-Tank Guard Filter','Base Size'))**Materials.get_value('Hold-Tank Guard Filter','Scaling Factor')
        ProductCIPBuffer=NumberOfProductTanks*(Protocol.get_value('Acid Buffer','Hold-Tanks')+Protocol.get_value('Caustic Buffer','Hold-Tanks'))*CorrectedProductTankVolume*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))
        ProductCIPBufferCost=ProductCIPBuffer*Materials.get_value('CIP Buffer','Base Cost')
        ProductHoldTankDiameter=NumberOfProductTanks*(CorrectedProductTankVolume*4000/(math.pi*Protocol.get_value('H/D tank ratio','Hold-Tanks')))**(1/3)
        ProductWFIVolume=(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*ProductHoldTankDiameter/100*Protocol.get_value('Water Flowrate (L/hr/m)','Hold-Tanks')/60*Protocol.get_value('WFI Rinse (min)','Hold-Tanks')
        ProductPWVolume=(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*ProductHoldTankDiameter/100*Protocol.get_value('Water Flowrate (L/hr/m)','Hold-Tanks')/60*Protocol.get_value('PW Rinse (min)','Hold-Tanks')
        ProductWFIandPWCost=ProductWFIVolume*Materials.get_value('WFI','Base Cost')+ProductPWVolume*Materials.get_value('PW','Base Cost')
           
        #Buffer Hold tanks, GFs, CIP, PW, WFI and Steam
        HoldTankVolumeList=[]
        HoldTankCostList=[]
        NumberOfTanksList=[]
        BufferCostList=[]
        GuardFiltersCostList=[]
        CIPBufferList=[]
        WFIVolumeList=[]
        PWVolumeList=[]
        CIPCostList=[]
        WFIandPWCostList=[]
        for x in PBChromBatchMode.Sizing(self)[6]:
            if x!=0:
                if x/Protocol.get_value('Space Efficiency','Hold-Tanks')>Equipment.get_value('Hold-Tank','Max. Size'):
                    NumberOfTanks=math.ceil((x/Protocol.get_value('Space Efficiency','Hold-Tanks'))/Equipment.get_value('Hold-Tank','Max. Size'))
                else:
                    NumberOfTanks=1
                HoldTankVolume=(x/Protocol.get_value('Space Efficiency','Hold-Tanks'))/NumberOfTanks
                if HoldTankVolume<=10:
                    CorrectedHoldTankVolume=math.ceil(HoldTankVolume)
                elif HoldTankVolume<=100 and HoldTankVolume>10:
                    CorrectedHoldTankVolume=int(math.ceil(HoldTankVolume/10.0))*10
                elif HoldTankVolume<=1000 and HoldTankVolume>100:
                    CorrectedHoldTankVolume=int(math.ceil(HoldTankVolume/100.0))*100
                else:
                    CorrectedHoldTankVolume=int(math.ceil(HoldTankVolume/1000.0))*1000
                HoldTankCost=NumberOfTanks*Equipment.get_value('Hold-Tank','Base Cost')*(CorrectedHoldTankVolume/Equipment.get_value('Hold-Tank','Base Size'))**Equipment.get_value('Hold-Tank','Scaling Factor')
                HoldTankGuard=NumberOfTanks*Materials.get_value('Hold-Tank Guard Filter','Base Cost')*(CorrectedHoldTankVolume/Materials.get_value('Hold-Tank Guard Filter','Base Size'))**Materials.get_value('Hold-Tank Guard Filter','Scaling Factor')
                CIPBuffer=NumberOfTanks*(Protocol.get_value('Acid Buffer','Hold-Tanks')+Protocol.get_value('Caustic Buffer','Hold-Tanks'))*CorrectedHoldTankVolume*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))
                CIPBufferCost=CIPBuffer*Materials.get_value('CIP Buffer','Base Cost')
                HoldTankDiameter=NumberOfTanks*(CorrectedHoldTankVolume*4000/(math.pi*Protocol.get_value('H/D tank ratio','Hold-Tanks')))**(1/3)
                WFIVolume=(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*HoldTankDiameter/100*Protocol.get_value('Water Flowrate (L/hr/m)','Hold-Tanks')/60*Protocol.get_value('WFI Rinse (min)','Hold-Tanks')
                PWVolume=(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*HoldTankDiameter/100*Protocol.get_value('Water Flowrate (L/hr/m)','Hold-Tanks')/60*Protocol.get_value('PW Rinse (min)','Hold-Tanks')
                WFIandPWCost=WFIVolume*Materials.get_value('WFI','Base Cost')+PWVolume*Materials.get_value('PW','Base Cost')
                BufferCost=x*Materials.get_value('Chromatography Buffers','Base Cost')
            else:
                CorrectedHoldTankVolume=0
                HoldTankDiameter=0
                HoldTankCost=0
                NumberOfTanks=0
                BufferCost=0
                HoldTankGuard=0
                CIPBuffer=0
                CIPBufferCost=0
                WFIVolume=0
                PWVolume=0
                WFIandPWCost=0
            HoldTankCostList.append(HoldTankCost)
            HoldTankVolumeList.append(CorrectedHoldTankVolume)
            NumberOfTanksList.append(NumberOfTanks)
            BufferCostList.append(BufferCost)
            GuardFiltersCostList.append(HoldTankGuard)
            CIPBufferList.append(CIPBuffer)
            WFIVolumeList.append(WFIVolume)
            PWVolumeList.append(PWVolume)
            CIPCostList.append(CIPBufferCost)
            WFIandPWCostList.append(WFIandPWCost)
        #Calculare total costs per batch
        TotalHoldTankCost=sum(HoldTankCostList)
        TotalBufferCost=sum(BufferCostList[0:8])+sum(BufferCostList[8:11])*PBChromBatchMode.Sizing(self)[4]/self.Batches
        TotalGuardFilterCost=sum(GuardFiltersCostList[0:8])+sum(GuardFiltersCostList[8:11])*PBChromBatchMode.Sizing(self)[4]/self.Batches
        TotalCIPCost=sum(CIPCostList[0:8])+sum(CIPCostList[8:11])*PBChromBatchMode.Sizing(self)[4]/self.Batches
        TotalWFIandPWCost=sum(WFIandPWCostList[0:8])+sum(WFIandPWCostList[8:11])*PBChromBatchMode.Sizing(self)[4]/self.Batches
        #Load Resin Database
        ResinData=pd.read_excel('Database.xlsx',sheetname='ResinList',index_col=0)
        #find Resin price for the selected resin
        ResinPrice=ResinData.get_value(self.ResinID,'Price (USD/L)')
        #Resin total cost and per batch
        TotalResinCost=ResinPrice*self.nCOL*PBChromBatchMode.Sizing(self)[0]*(1+Protocol.get_value('Resin Overfill','Chromatography'))*PBChromBatchMode.Sizing(self)[4]
        ResinCostPerBatch=TotalResinCost/self.Batches        
        #Check skid availability
        if Flowrate<=Equipment.get_value('Chromatography Skid','Max. Size'):
            #Chromatography skid
            SkidCost=self.nCOL*Equipment.get_value('Chromatography Skid','Base Cost')*((int(math.ceil(Flowrate/10.0))*10)/Equipment.get_value('Chromatography Skid','Base Size'))**Equipment.get_value('Chromatography Skid','Scaling Factor')
            #Check column technology                                                               
            if self.Disposable==False:
                #Self-packed Chromatography column
                SelfpackedColumnCost=self.nCOL*Equipment.get_value('Chromatography Column','Base Cost')*(PBChromBatchMode.Sizing(self)[1]/Equipment.get_value('Chromatography Column','Base Size'))**Equipment.get_value('Chromatography Column','Scaling Factor')
                #Empty Pre-packed column
                PrepackedEmptyColumnCost=0
                #Select packing system
                if Flowrate<=Equipment.get_value('Packing System (Small)','Max. Size'):
                    PackingSystemCost=self.nCOL*Equipment.get_value('Packing System (Small)','Base Cost')
                elif Flowrate<=Equipment.get_value('Packing System (Big)','Max. Size') and Flowrate>=Equipment.get_value('Packing System (Big)','Min. Size'):
                    PackingSystemCost=self.nCOL*Equipment.get_value('Packing System (Big)','Base Cost')
            else:
                SelfpackedColumnCost=0
                PackingSystemCost=0
                PrepackedEmptyColumnCost=self.nCOL*PBChromBatchMode.Sizing(self)[1]*PBChromBatchMode.Sizing(self)[4]*Materials.get_value('Empty Pre-packed Column','Base Cost')/self.Batches
            return SkidCost,SelfpackedColumnCost,PackingSystemCost,TotalHoldTankCost,TotalBufferCost,TotalCIPCost,TotalWFIandPWCost,ResinCostPerBatch,PrepackedEmptyColumnCost,TotalGuardFilterCost
        else:
            return "Flowrate too high, there is no available skid"
    
    def CostBreakdown(self):
        EquipmentPurchaseCost=sum(PBChromBatchMode.EquipmentAndMaterials(self)[0:4])
        #costs per batch
        ChemicalReagentsCost=sum(PBChromBatchMode.EquipmentAndMaterials(self)[4:7])
        ConsumablesCost=sum(PBChromBatchMode.EquipmentAndMaterials(self)[7:10])
        LabourCost=0
        return LabourCost,ChemicalReagentsCost,ConsumablesCost,EquipmentPurchaseCost


In [2]:
Capture=PBChromBatchMode(MassIn=10,VolumeIn=1000,BedHeight=20,Batches=20,Disposable=False,LinVeloc=300,nCOL=1,nCYC=5,ResinID=2,TrainPosition=1)
print(Capture.Sizing())
print(Capture.MassBalance())
print(Capture.Timeframe())
print(Capture.EquipmentAndMaterials())
print(Capture.CostBreakdown())

(62.486277879900989, 60, 22.100000000000001, 4.0, 1, 40, [1149.7475129901782, 1149.7475129901782, 574.87375649508908, 862.31063474263362, 862.31063474263362, 862.31063474263362, 287.43687824754454, 215.57765868565841, 1006.0290738664058, 1437.1843912377228, 359.29609780943071])
(9.5, 749.83533455881184, 12.669448293723862, 3.0, 0.94999999999999996)
(30.0, 2.1786666666666665, 2.504666666666667, 5.5986666666666665, 0.51566666666666672, 0.36833333333333335, 6.0)
(254669.90376350578, 66500.0, 35000, 634316.41076745105, 12208.881403564455, 40451.25, 2258.018044271761, 55847.110855161518, 0, 2930.0443567865964)
(0, 54918.149447836215, 58777.155211948113, 990486.31453095679)
